<div style="
font-size: 28px;
font-weight: bold;
text-align: center;
margin-top: 20px;
margin-bottom: 20px;
color: #2c3e50;
">

Conversion d'un ensemble de règles en automate asynchrone,
dans différents formats

</div>


# 0. Bibliothèque à importer

In [ ]:
import pylfit
from random import randint
import libsbml
import pypint

# I. Le format adapté à Pint

La fonction `modelToPint` ci-dessous permet, à partir d’un modèle (composé d’un ensemble de features, de target et de rules),  
de générer l’automate asynchrone correspondant, au format .an, compatible avec Pint.

Cette fonction permet de traiter les modèles booléens comme multi-valuées.

On pars aussi du principe que toutes les valuers sont des chaînes de caractères, et non pas seulement 0 et 1 (pour le cas booléen). Si cela n'est pas souhaité, il est possible de modifier 2 - 3 lignes de la fonction pour revenir au cas entier.

L'idée globale de cette fonction (et de toute les suivantes) et de récupérer les éléments nécessaires à l'écriture du document, puis de les écrire avec une certaine syntaxe dans un document de sortie.

In [4]:
def modelToPint(model, outputName = "pint"):
    with open(f"{outputName}.an", "w") as f:
        for features in model.features:
            f.write(f"\"{features[0][:-4]}\" {features[1]}\n")
        f.write("\n")
        
        for rules in model.rules:
            headVariable = rules.head.variable[:-2]
            bodyValues = rules.body.values()
            lenBodyValues = len(bodyValues)
            listBodyValues = list(bodyValues)
            bodyVariables = [values.variable[:-4] for values in bodyValues]
            
            if headVariable not in bodyVariables:
                etats = model.features[[model.features[i][0][:-4] for i in range(len(model.features))].index(headVariable)][1]
                for etat in etats:
                    if (etat != rules.head.value):
                        f.write(f"\"{rules.head.variable[:-2]}\" \"{etat}\" -> \"{rules.head.value}\" when ")
                        for condition in range(lenBodyValues):
                            f.write(f"\"{listBodyValues[condition].variable[:-4]}\"=\"{listBodyValues[condition].value}\"")
                            if condition < lenBodyValues - 1:
                                f.write(" and ")
                        f.write("\n")    
            
            else:
                nextValue = rules.body[rules.head.variable+"_1"].value 
                if nextValue != rules.head.value:
                    if lenBodyValues == 1:
                        f.write(f"\"{rules.head.variable[:-2]}\" \"{nextValue}\" -> \"{rules.head.value}\"\n")
                    else:
                        f.write(f"\"{rules.head.variable[:-2]}\" \"{nextValue}\" -> \"{rules.head.value}\" when ")
                        for condition in range(lenBodyValues):
                            currentVar = listBodyValues[condition].variable[:-4]
                            if (currentVar != headVariable):
                                f.write(f"\"{listBodyValues[condition].variable[:-4]}\"=\"{listBodyValues[condition].value}\"")
                                if condition < lenBodyValues - 1:
                                    f.write(" and ")
                        f.write("\n")

# II. Le format Pyboolnet

De la même façon, la fonction `modelToPyboolnet` ci-dessous permet de traduire un modèle 
au format .bnet.
<p>


In [3]:
def modelToPyboolnet(model, outputName = "pyboolnet"):
    variable = [i[0][:-2] for i in  model.targets]
    rules = model.rules
    with open(f"{outputName}.bnet", "w") as f:
        for var in variable: 
            countOrRules = 0
            f.write(f"{var}, ")
            for rule in rules:
                countAndRules = 0
                if rule.head.variable[:-2] == var and rule.head.value == "1":
                    listBody = list(rule.body.values())
                    lenBody = len(listBody)
                    if countOrRules != 0:
                        f.write(" | ")
                    if lenBody > 1:
                        f.write("(")
                    for condition in listBody:
                        if countAndRules != 0:
                            f.write(" & ")
                        if condition.value == "0":
                            f.write(f"!{condition.variable[:-4]}")
                        else:
                            f.write(f"{condition.variable[:-4]}")
                        countAndRules += 1
                    if lenBody > 1:
                        f.write(")")
                    countOrRules += 1
                #elif rule.head.variable[:-2] != var:
                    #break
            if countOrRules == 0:
                f.write("1")
            f.write("\n")

La fonction ci-dessous permet de "boucler" le processus (fonction déjà implémenté par Tony Ribeiro).

In [ ]:
def bouclePyboolnet(file):
    model = pylfit.preprocessing.boolean_network.dmvlp_from_boolean_network_file(file)
    model.summary()

# III. Le format Ginml, adapté à Ginsim

La fonction ci-dessous permet de traduire un modèle 
au format .ginml, utilisable ensuite dans Ginsim. Attention, cette première version ne convient que pour les modèles booléens.
<p>


In [ ]:
def modelToGinml(model, outputFile = "ginml"):
    variablesAndMax = [(i[0][:-4], i[1][-1]) for i in model.features]
    variables = [i[0] for i in variablesAndMax]
    rules = model.rules
    text = {}
    value = {}

    with open(f"{outputFile}.ginml", 'w') as f:
        f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        f.write('<!DOCTYPE gxl SYSTEM "http://ginsim.org/GINML_2_2.dtd">\n<gxl xmlns:xlink="http://www.w3.org/1999/xlink">\n')
        f.write(f'\t<graph id="phage4" class="regulatory" nodeorder="{" ".join(variables)}">\n')
        f.write(f'\t\t<nodestyle background="#ffffff" foreground="#000000" text="#000000" shape="RECTANGLE" width="45" height="25" properties="intermediate:#ffff00 active:#ffc800"/>\n\t\t<edgestyle color="#000000" pattern="SIMPLE" line_width="1" properties="positive:#00c800 negative:#c80000 dual:#0000c8"/>\n\n')
        for var in variablesAndMax:
            text[var[0]] = f'\t\t<node id="{var[0]}" maxvalue="{var[1]}">\n'
        
        edge = ""
        for var in variables: 
            put = False
            value[var] = '\t\t\t<value val="1">\n'
            for rule in rules:
                if rule.head.variable[:-2] == var and rule.head.value == "1":
                    
                    listBody = list(rule.body.values())
                    lenBody = len(listBody)
                    name = ""
                    for condition in listBody:
                        if condition.value == "0":
                            edge += f'\t\t<edge id="{var}:{condition.variable[:-4]}" from="{var}" to="{condition.variable[:-4]}" sign="negative">\n\t\t\t<edgevisualsetting anchor="NE" style=""/>\n\t\t</edge>\n'
                            if name == "":
                                name = f'\t\t\t\t<exp str="!{condition.variable[:-4]}'
                            else:
                                name += f' &amp !{condition.variable[:-4]}'
                            put = True
                        else:
                            if f'from="{var}" to="{condition.variable[:-4]}" sign="positive"' not in edge:
                                edge += f'\t\t<edge id="{var}:{condition.variable[:-4]}" from="{var}" to="{condition.variable[:-4]}" sign="positive">\n\t\t\t<edgevisualsetting anchor="NE" style=""/>\n\t\t</edge>\n'
                            if name == "":
                                name = f'\t\t\t\t<exp str="{condition.variable[:-4]}'
                            else:
                                name += f' &amp; {condition.variable[:-4]}'
                            put = True
                        if f'{var}:{condition.variable[:-4]}' not in text[var]:
                            text[var] += f'\t\t\t<parameter idActiveInteractions=" {var}:{condition.variable[:-4]}" val="1"/>\n' 
                    name += f'"/>\n'
                    value[var] += name
            if put:
                value[var] += '\t\t\t</value>\n'
                text[var] += value[var] 
            
        
        for node in list(text.values()):
            f.write(f'{node}\t\t\t<nodevisualsetting x="{randint(100,600)}" y="{randint(100,600)}" style=""/>\n\t\t</node>\n\n')
        
        f.write(f'{edge}\n\t</graph>\n</gxl>')

# IV. Le format Sbml-quad

In [ ]:
def formuleLogic(model, var0):
    rules = model.rules
    orClauses = []

    for rule in rules:
        if rule.head.variable[:-2] == var0 and rule.head.value == "1":
            andConditions = []
            for condition in rule.body.values():
                var = condition.variable[:-4]
                if condition.value == "0":
                    andConditions.append(f"!{var}")
                else:
                    andConditions.append(var)
            
            clause = "(" + " && ".join(andConditions) + ")"
            orClauses.append(clause)

    if not orClauses:
        return "0"
    
    return " || ".join(orClauses)

def modelToSbmlQual(model, outputFile="sbml"):
    doc = libsbml.SBMLDocument(3, 1)
    model_sbml = doc.createModel()
    qmodel = model_sbml.getPlugin("qual")

    variables_info = [(i[0][:-4], i[1][-1]) for i in model.features]
    for name, max_val in variables_info:
        qs = qmodel.createQualitativeSpecies()
        qs.setId(name)
        qs.setConstant(False)
        qs.setMaxLevel(int(max_val))

    for name, _ in variables_info:
        transition = qmodel.createTransition()
        
        output = transition.createOutput()
        output.setQualitativeSpecies(name)
        output.setTransitionEffect(libsbml.OUTPUT_TRANSITION_EFFECT_ASSIGNMENT_LEVEL)

        logic_str = formuleLogic(model, name)

        for input_name, _ in variables_info:
            if input_name in logic_str and input_name != name:
                input_obj = transition.createInput()
                input_obj.setQualitativeSpecies(input_name)
                input_obj.setTransitionEffect(libsbml.INPUT_TRANSITION_EFFECT_NONE)

        ast_node = libsbml.parseL3Formula(logic_str)
        ft = transition.createFunctionTerm()
        ft.setResultLevel(1) 
        ft.setMath(ast_node)
        
        default_term = transition.createDefaultTerm()
        default_term.setResultLevel(0)

    libsbml.writeSBMLToFile(doc, f'{outputFile}.sbml')

# IV. Test et résultats

L'ensemble des fonctions ci-dessus ont été testé sur un exemple.

Il s'agit de celui présenté dans le fichier README.me de pylfit, dont on récupère le modèle de la manière suivante :

In [ ]:
data = [ \
(["0","0","0"],["0","0","1"]), \
(["1","0","0"],["0","0","0"]), \
(["0","1","0"],["1","0","1"]), \
(["0","0","1"],["0","0","1"]), \
(["1","1","0"],["1","0","0"]), \
(["1","0","1"],["0","1","0"]), \
(["0","1","1"],["1","0","1"]), \
(["1","1","1"],["1","1","0"])]

dataset = pylfit.preprocessing.discrete_state_transitions_dataset_from_array(data=data, feature_names=["p_t_1","q_t_1","r_t_1"], target_names=["p_t","q_t","r_t"])

model = pylfit.models.DMVLP(features=dataset.features, targets=dataset.targets)
model.compile(algorithm="pride") 

model.fit(dataset=dataset)

Il s'agit ensuite d'ouvrir les différents fichiers de sortis avec un logiciel adapté, de récupérer le graphe de sorti pour comparer les différents résultats, et espérer optenir la même chose pour un même modèle.

Pour les formats PyBoolnet, Ginml et SBML-quad, on utilise Ginsim.

PyBoolnet et SBML-quad : <br><img src="graphe/ex1/bnet.png" style="height:200px;display: block;margin: 0 auto">

Ginml version 1 : <br><img src="graphe/ex1/ginml1.png" style="height:200px;display: block;margin: 0 auto">

Pour visualiser le graphe à partir du fichier .an, on utilise le module pypint.

Pint : 

# V. Bibliographie

- https://ginsim.github.io/documentation/ 
- https://colomoto.github.io/formats/ginml/
- https://colomoto.github.io/formats/sbml-qual/
- https://github.com/sbmlteam/libsbml
- https://en.wikipedia.org/wiki/LibSBML
- https://sbml.org/software/libsbml/5.18.0/docs/formatted/python-api/classlibsbml_1_1_s_base_plugin.html
- https://sbml.org/software/libsbml/5.18.0/docs/formatted/python-api/classes.html
- https://github.com/Tony-sama/pylfit
- https://loicpauleve.name/pint/doc/automata-networks.html